In [1]:
%pip install roboflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key="xR2Y9RF65j4EqhMqb5QM")
project = rf.workspace("treyder38").project("_seg-2li56")
version = project.version(19)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to ЛЦТ_seg-19 in yolov11:: 100%|██████████| 15942/15942 [04:38<00:00, 57.19it/s] 


In [4]:
import os
from pathlib import Path
from typing import List, Tuple
import math

# ===== настройки =====
DATASET_DIR = Path("/home/jupyter/datasphere/project/ЛЦТ_seg-19")  # папка, где лежит data.yaml и каталоги train/valid/test
SPLITS = ["train", "valid"]
# =====================

def read_seg_file(path: Path) -> List[Tuple[int, List[Tuple[float, float]]]]:
    """
    Читает yolo-seg label-файл.
    Возвращает список (cls, [(x,y), ...]) — одна запись на ОДИН сегмент.
    """
    segs = []
    if not path.exists():
        return segs
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            try:
                cls = int(float(parts[0]))
            except ValueError:
                # пропускаем битые строки
                continue
            nums = [float(x) for x in parts[1:]]
            # берём попарно, игнорируя неполный хвост
            pts = []
            for i in range(0, len(nums) - 1, 2):
                x, y = nums[i], nums[i + 1]
                # защитимся от выхода за [0,1]
                x = min(max(x, 0.0), 1.0)
                y = min(max(y, 0.0), 1.0)
                pts.append((x, y))
            if len(pts) >= 3:  # минимум треугольник
                segs.append((cls, pts))
    return segs

def seg_to_xyxy(pts: List[Tuple[float, float]]) -> Tuple[float, float, float, float]:
    xs = [p[0] for p in pts]
    ys = [p[1] for p in pts]
    return min(xs), min(ys), max(xs), max(ys)

def xyxy_to_xywhn(x1, y1, x2, y2):
    w = max(0.0, x2 - x1)
    h = max(0.0, y2 - y1)
    cx = x1 + w / 2.0
    cy = y1 + h / 2.0
    return cx, cy, w, h

def merge_boxes_xyxy(boxes):
    # Всегда объединяем ВСЕ боксы класса в один охватывающий
    if not boxes:
        return []
    x1 = min(b[0] for b in boxes)
    y1 = min(b[1] for b in boxes)
    x2 = max(b[2] for b in boxes)
    y2 = max(b[3] for b in boxes)
    # на всякий случай зажмём в [0,1]
    x1 = max(0.0, min(1.0, x1))
    y1 = max(0.0, min(1.0, y1))
    x2 = max(0.0, min(1.0, x2))
    y2 = max(0.0, min(1.0, y2))
    return [(x1, y1, x2, y2)]

def convert_split(split_dir: Path):
    src_labels = split_dir / "labels"
    out_labels = split_dir / "labels"
    out_labels.mkdir(parents=True, exist_ok=True)

    txts = sorted([p for p in src_labels.glob("*.txt") if p.is_file()])
    for lp in txts:
        segs = read_seg_file(lp)
        # группируем bbox'ы по классу
        per_class = {}
        for cls, pts in segs:
            x1, y1, x2, y2 = seg_to_xyxy(pts)
            # фильтр нулевых
            if x2 <= x1 or y2 <= y1:
                continue
            per_class.setdefault(cls, []).append((x1, y1, x2, y2))

        # сливаем разрозненные сегменты одного класса
        final_boxes = []
        for cls, boxes in per_class.items():
            merged = merge_boxes_xyxy(boxes)
            for (x1, y1, x2, y2) in merged:
                cx, cy, w, h = xyxy_to_xywhn(x1, y1, x2, y2)
                # защита от нулевого размера
                if w <= 0 or h <= 0:
                    continue
                final_boxes.append((cls, cx, cy, w, h))

        # пишем файл детекции
        outp = out_labels / lp.name
        with outp.open("w", encoding="utf-8") as f:
            for cls, cx, cy, w, h in final_boxes:
                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    print(f"[OK] {split_dir.name}: {len(txts)} файлов → {out_labels}")

def main():
    for split in SPLITS:
        d = DATASET_DIR / split
        if d.exists():
            convert_split(d)
        else:
            print(f"[skip] нет директории: {d}")

if __name__ == "__main__":
    main()

[OK] train: 7419 файлов → /home/jupyter/datasphere/project/ЛЦТ_seg-19/train/labels
[OK] valid: 548 файлов → /home/jupyter/datasphere/project/ЛЦТ_seg-19/valid/labels
